## Get Nodes from OSM

In [1]:
import pandas as pd
import urllib.request
import xml.etree.ElementTree as ET

# berlin
# boundingbox=[12.8389,52.3030,13.9348,52.7246]
# charlottenburg
# boundingbox = [13.2457, 52.5045, 13.3470, 52.5288]
# westend
boundingbox = [13.2521, 52.5123, 13.2820, 52.5237]
# r= requests.get("http://overpass-api.de/api/map?bbox=13.2457,52.5045,13.3470,52.5288")
#TODO URL an boundingbox anpassen
url="https://overpass-api.de/api/interpreter?data=node%5B%22surface%22%5D%2852%2E494853137273%2C13%2E242473602295%2C52%2E538935532664%2C13%2E321695327759%29%3Bway%5B%22surface%22%5D%5B%22highway%22%5D%2852%2E494853137273%2C13%2E242473602295%2C52%2E538935532664%2C13%2E321695327759%29%3B%28%2E%5F%3B%3E%3B%29%3Bout%3B%0A"

get Data

In [2]:

response = urllib.request.urlopen(url)
xml = response.read()
response.close()

parse Data

In [3]:
root = ET.fromstring(xml)
#root = tree.getroot()


create surface Dataframe

In [4]:

nodes_surface = list()
for way in root.findall('way'):
    for child in list(way):
        if(child.attrib.get('k')=='surface'):
            #print(child.attrib.get('v'))
            for node in way.findall('nd'):
                nodes_surface.append([child.attrib.get('v'), way.attrib.get('id'), node.attrib.get('ref')])
                
df_surface_node=pd.DataFrame(nodes_surface,columns=['surface','wayId','nodeId'])
df_surface_node

,surface,wayId,nodeId
0,asphalt,4067882,27560124
1,asphalt,4067882,282898608
2,asphalt,4067882,282898609
3,asphalt,4067882,16541461
4,asphalt,4067883,29030447
...,...,...,...
13561,asphalt,732746880,28381832
13562,asphalt,732746880,6861384782
13563,paving_stones,734251578,6876083160
13564,paving_stones,734251578,6876083162


create list of node

In [5]:
node_list = list()
for way in root.findall('node'):
    node_list.append([way.attrib.get('id'), way.attrib.get('lat'), way.attrib.get('lon')])
df_nodes = pd.DataFrame(node_list,columns=['nodeId','lat','lon'])
df_nodes

,nodeId,lat,lon
0,458322,52.5037272,13.2823718
1,458323,52.5042930,13.2827047
2,458324,52.5051202,13.2828139
3,458325,52.5054972,13.2828394
4,458326,52.5062206,13.2830150
...,...,...,...
10338,6923867130,52.5147673,13.2645672
10339,6924044364,52.4948849,13.2880992
10340,6924047886,52.5173045,13.2552476
10341,6924471471,52.5136978,13.2603891


In [6]:
df_joined_list = df_surface_node.merge(df_nodes,on='nodeId',how='left')

In [7]:
df_joined_list

,surface,wayId,nodeId,lat,lon
0,asphalt,4067882,27560124,52.5153490,13.2493683
1,asphalt,4067882,282898608,52.5154934,13.2496738
2,asphalt,4067882,282898609,52.5155538,13.2498976
3,asphalt,4067882,16541461,52.5156244,13.2502680
4,asphalt,4067883,29030447,52.5077092,13.2615167
...,...,...,...,...,...
13561,asphalt,732746880,28381832,52.5389616,13.2703617
13562,asphalt,732746880,6861384782,52.5388520,13.2702783
13563,paving_stones,734251578,6876083160,52.5088665,13.3121038
13564,paving_stones,734251578,6876083162,52.5089771,13.3121226


In [8]:
#count nodes per way
df_surface_node.groupby('wayId').count()

,surface,nodeId
wayId,,
102698235,5,5
102943813,2,2
102943816,2,2
102945288,10,10
103258836,11,11
...,...,...
9873997,16,16
9874982,2,2
9874983,4,4


## Download pictures for each node

In [9]:
import urllib
import json
import os
import shutil
import argparse
import time

In [22]:
BASE_DIR = '/home/stefan/git/OSMMapillary/downloaded/'
# See https://www.mapillary.com/developer/api-documentation/

MAPILLARY_API_IM_SEARCH_URL = 'https://a.mapillary.com/v3/images?'
MAPILLARY_API_IM_RETRIEVE_URL = 'https://d1cuyjsrcm0gby.cloudfront.net/'
CLIENT_ID = 'TG1sUUxGQlBiYWx2V05NM0pQNUVMQTo2NTU3NTBiNTk1NzM1Y2U2'
nrOfImageDownloadsPerNode=5

In [23]:

'''
Script to download images using the Mapillary image search API.

Downloads images inside a rect (min_lat, max_lat, min_lon, max_lon).
'''

def create_dirs(base_path):
    try:
        shutil.rmtree(base_path)
    except:
        pass
    os.mkdir(base_path)


def query_search_api(lon, lat, max_results):
    '''
    Send query to the search API and get dict with image data.
    '''

    # Create URL
    params = urllib.parse.urlencode(list(zip(
        ['client_id','closeto','per_page'],
        [CLIENT_ID, ','.join([str(lon), str(lat)]), str(max_results)])), doseq=True)
    
    print(MAPILLARY_API_IM_SEARCH_URL + params)

    # Get data from server, then parse JSON
    query = urllib.request.urlopen(MAPILLARY_API_IM_SEARCH_URL + params).read()
    query = json.loads(query)['features']

    print("Result: {0} images in area.".format(len(query)))
    return query


def download_images(query, path, size=640):
    '''
    Download images in query result to path.

    Return list of downloaded images with lat,lon.
    There are four sizes available: 320, 640, 1024 (default), or 2048.
    '''
    im_size = "thumb-{0}.jpg".format(size)
    im_list = []

    for im in query:
        # Use key to create url to download from and filename to save into
        key = im['properties']['key']
        url = MAPILLARY_API_IM_RETRIEVE_URL + key + '/' + im_size
        filename = key + ".jpg"

        #try:
        # Get image and save to disk
        if(os.path.isfile(path+filename)==False):
            image = urllib.request.urlretrieve(url, path + filename)
            #image.retrieve(url, path + filename)

            # Log filename and GPS location
            coords = ",".join(map(str, im['geometry']['coordinates']))
            im_list.append([filename, coords])

            print("Successfully downloaded: {0}".format(filename))
        else:
            print("file "+filename+" already exists" )
       # except KeyboardInterrupt:
       #     break
       # except:
       #     print("Failed to download: {0}".format(filename))
    
    return im_list


#if __name__ == '__main__':
    '''
    Use from command line as below, or run query_search_api and download_images
    from your own scripts.
    '''

    parser = argparse.ArgumentParser()
    parser.add_argument('min_lat', type=float)
    parser.add_argument('max_lat', type=float)
    parser.add_argument('min_lon', type=float)
    parser.add_argument('max_lon', type=float)
    parser.add_argument('--max_results', type=int, default=400)
    parser.add_argument('--image_size', type=int, default=1024, choices=[320,640,1024,2048])
    args = parser.parse_args()

    # query api
    query = query_search_api(args.min_lat, args.max_lat, args.min_lon, args.max_lon, args.max_results)

    # create directories for saving
    create_dirs(BASE_DIR)

    # download
    downloaded_list = download_images(query, path=BASE_DIR, size=args.image_size)

    # save filename with lat, lon
    with open(BASE_DIR+"downloaded.txt", "w") as f:
        for data in downloaded_list:
            f.write(",".join(data) + "\n")


In [24]:
for i in range(0, len(df_joined_list)):
    print("Currently working on Row "+str(i))
    #time.sleep(1)
    #check if dir exists
    path = BASE_DIR + df_joined_list.loc[i]['surface'] + "/"
    if(os.path.isdir(path)==False):
        create_dirs(path)
        print("created dir " + path)
    query=query_search_api(df_joined_list.loc[i]['lon'],df_joined_list.loc[i]['lat'],nrOfImageDownloadsPerNode)
    download_images(query,path)

Currently working on Row 0
https://a.mapillary.com/v3/images?client_id=TG1sUUxGQlBiYWx2V05NM0pQNUVMQTo2NTU3NTBiNTk1NzM1Y2U2&closeto=13.2493683%2C52.5153490&per_page=5
Result: 5 images in area.
file 7OBKWjJaXC6MkPfaxVcisw.jpg already exists
file EWNtUHODip4ITi38NUfmcA.jpg already exists
file 6SMPLNzEXau43fIvEcKa7w.jpg already exists
file a6r1ZHNx0K_2X-EG8eNi5A.jpg already exists
file H2xpWrznsnizc4wc6VikwQ.jpg already exists
Currently working on Row 1
https://a.mapillary.com/v3/images?client_id=TG1sUUxGQlBiYWx2V05NM0pQNUVMQTo2NTU3NTBiNTk1NzM1Y2U2&closeto=13.2496738%2C52.5154934&per_page=5
Result: 5 images in area.
Successfully downloaded: drYzoDOC0iipv7JMJ9AmUQ.jpg
Successfully downloaded: KKMOOHCuDJdu8mDn5CMmCQ.jpg
Successfully downloaded: gLGRgEd0IisU1n9mIGbcKQ.jpg
Successfully downloaded: KE6gSu2tRxMq7ZyXonzF5A.jpg
Successfully downloaded: 5Le6xVO80h47bj2-7d6ueg.jpg
Currently working on Row 2
https://a.mapillary.com/v3/images?client_id=TG1sUUxGQlBiYWx2V05NM0pQNUVMQTo2NTU3NTBiNTk1NzM1

KeyboardInterrupt: 

,surface,wayId,nodeId,lat,lon
0,cobblestone,4396012,26682488,52.5053376,13.2937720
1,cobblestone,4396012,26736222,52.5044765,13.2925809
2,cobblestone,4436949,26731269,52.5034509,13.2954875
3,cobblestone,4436949,370630001,52.5042432,13.2990022
4,cobblestone,4471968,26757189,52.5300656,13.3132040
5,cobblestone,4471968,297959552,52.5296661,13.3133410
6,cobblestone,4471968,297959540,52.5287027,13.3136718
7,cobblestone,4471968,2025641950,52.5285750,13.3137178
8,cobblestone,4471968,297959845,52.5282331,13.3138409
9,cobblestone,4471968,3305073679,52.5279545,13.3139456


In [127]:
for i in range(0, len(df_surface_cobblestone)):
#for i in range(1, 10):
    #check if dir exists
    path = BASE_DIR + df_surface_cobblestone.loc[i]['surface'] + "/"
    if(os.path.isdir(path)==False):
        create_dirs(path)
        print("created dir " + path)
    query=query_search_api(df_surface_cobblestone.loc[i]['lon'],df_surface_cobblestone.loc[i]['lat'],10)
    download_images(query,path)

https://a.mapillary.com/v3/images?client_id=TG1sUUxGQlBiYWx2V05NM0pQNUVMQTo2NTU3NTBiNTk1NzM1Y2U2&closeto=13.2925809%2C52.5044765&per_page=10
Result: 10 images in area.
Successfully downloaded: Ae39ORnJNGY_hNXInRm9tA.jpg
Successfully downloaded: zAYnsK3exKqlyL6h4Owo-A.jpg
Successfully downloaded: aAqIeKi_ZzrF1rGaMhGqMA.jpg
Successfully downloaded: j-waM29whiI5UvWa_OfKBg.jpg
Successfully downloaded: _mIOd63quMsPOTrfbdVFBg.jpg
Successfully downloaded: PmoDhOUK_1yDtNvPcUmttw.jpg
Successfully downloaded: R7pO82SI0cMHXcBIFMIHoQ.jpg
Successfully downloaded: HeeiWx6JODOpYdtAl8jbRA.jpg
Successfully downloaded: cLGcnALIh2mRmHfPwEdN3A.jpg
Successfully downloaded: avHWw3Ijo64R3MEWNYvuNg.jpg
https://a.mapillary.com/v3/images?client_id=TG1sUUxGQlBiYWx2V05NM0pQNUVMQTo2NTU3NTBiNTk1NzM1Y2U2&closeto=13.2954875%2C52.5034509&per_page=10
Result: 10 images in area.
Successfully downloaded: s2lHhmCiPT3nf6ZfdpD2Ow.jpg
Successfully downloaded: G0YtyBOsjSODTDsIc8SAoQ.jpg
Successfully downloaded: QVoAxEi0XBr5RFV

,index,surface,wayId,nodeId,lat,lon
0,246,cobblestone,4396012,26682488,52.5053376,13.2937720
1,247,cobblestone,4396012,26736222,52.5044765,13.2925809
2,653,cobblestone,4436949,26731269,52.5034509,13.2954875
3,654,cobblestone,4436949,370630001,52.5042432,13.2990022
4,1036,cobblestone,4471968,26757189,52.5300656,13.3132040
5,1037,cobblestone,4471968,297959552,52.5296661,13.3133410
6,1038,cobblestone,4471968,297959540,52.5287027,13.3136718
7,1039,cobblestone,4471968,2025641950,52.5285750,13.3137178
8,1040,cobblestone,4471968,297959845,52.5282331,13.3138409
9,1041,cobblestone,4471968,3305073679,52.5279545,13.3139456


range(0, 448)